In [1]:

import time
import serial
import socket
#from socket import *

import subprocess
import multiprocessing
import pynq.lib.rgbled as rgbled
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary


from pynq.lib.arduino import Arduino_Analog
from pynq.lib.arduino import ARDUINO_GROVE_A1
from pynq.lib.arduino import arduino_io

base = BaseOverlay("base.bit")
analog1 = Arduino_Analog(base.ARDUINO,ARDUINO_GROVE_A1)


In [2]:
liba = MicroblazeLibrary(base.PMODA, ['i2c'])

In [74]:
%%microblaze base.PMODA

#include "i2c.h"

int read_Temp() {
    unsigned int low_byte; 
    unsigned int high_byte; 
   
    i2c device = i2c_open(3, 2);
    unsigned char buf[2];
    buf[0] = 0;
    
    i2c_write(device, 0x5A, buf, 1); // Write to Device Address
    i2c_write(device, 0x07, buf, 1); // Write to Register Address
   
    high_byte = i2c_read(device, 0x5A, buf, 1); // Read Device Address
    low_byte = i2c_read(device, 0x07, buf, 1); // Read Register Address

    return (high_byte+low_byte); // Concat 2 Bytes
    //return ((buf[0] & 0x0F) << 8) | buf[1];
}


In [75]:
temp = read_Temp()
print(temp)

-4


In [4]:

manager = multiprocessing.Manager()
seq_complete_list = manager.list()
serverName = 'ec2-35-90-59-237.us-west-2.compute.amazonaws.com' # AWS Server Public DNS

def get_temp():
    temp = 0
    serverName = 'ec2-35-90-59-237.us-west-2.compute.amazonaws.com' # AWS Server Public DNS
    clientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
    
    while True:
        temp = read_Temp()
        print(str(temp))
        clientSocket.sendto(str(int(temp)).encode(),(serverName, 1117))
        time.sleep(0.05)
        if len(seq_complete_list) > 0:
            clientSocket.close()
            break
        else:
            time.sleep(5)
            temp = 0
        clientSocket.close()


def main():
    # Create 1 process
    p1 = multiprocessing.Process(target=get_temp)

    # Start process
    p1.start()
    for t in range(10):
        time.sleep(2)
    seq_complete_list.append(1)
    
    # Give sufficient time for socket to close 
    time.sleep(2)
    
    # Join process
    p1.join()
    p1.close() # wait for process1 to finish
    
    print('Sequence Stopped!')

if __name__ == '__main__':
    main()

2
2


Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_860/789779177.py", line 15, in get_temp
    clientSocket.sendto(str(int(temp)).encode(),(serverName, 1117))
OSError: [Errno 9] Bad file descriptor


Sequence Stopped!
